In [1]:
from bs4 import BeautifulSoup
import requests
from lxml import html
from urllib.parse import urljoin
from selenium import webdriver
from urllib.request import urlretrieve
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import warnings
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait   
from selenium.webdriver.support import expected_conditions as EC
import csv
warnings.filterwarnings('ignore')
options=Options()

options.add_extension('../driver/adblock.crx')
options.add_argument('--start-maximized')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
#options.add_argument('--incognito')
#options.add_argument('user-data-dir=selenium')

In [2]:
PATH = ChromeDriverManager().install()

In [3]:
driver = webdriver.Chrome(PATH, options=options)

In [4]:
url = 'https://www.moto1pro.com/'

In [5]:
driver.get(url)

In [6]:
# Acepto las cookies
cookies = driver.find_element(By.XPATH, '/html/body/div[11]/div[2]/div[1]/div[2]/div[2]/button[1]/p')
cookies.click()
time.sleep(0.5)

In [7]:
#Accedo a las fichas tecnicas
fichas = driver.find_elements(By.XPATH, '//*[@id="navWrapper"]/li[6]/a')
fichas[0].click()
time.sleep(0.5)

In [8]:
elems = driver.find_elements(By.XPATH, "//a[@href]")

In [9]:
links = []
for elem in elems:
    links.append(elem.get_attribute("href"))

In [10]:
links = [elem.get_attribute("href") for elem in elems]

In [11]:
links_marcas = []             
for e in links:
    if 'marca-de-motos/' in e:
        links_marcas.append(e)
#links_marcas

In [12]:
url_base = "https://www.moto1pro.com/marca-de-motos/"                                    
marcas = ['Aprilia', 'Bmw', 'Ducati', 'Honda', 'Yamaha', 'Kawasaki', 'Suzuki', 'KTM']

full_links = []                                                           
visited_links = set()

for marca in marcas:
    url = urljoin(url_base, marca)
    response = requests.get(url)
    tree = html.fromstring(response.content)
    div_content = tree.xpath('//div[@class="contenido-izquierda"]')[0]
    links = div_content.xpath('.//a/@href')
    for link in links:
        full_link = urljoin(url, link)
        if full_link not in visited_links:
            full_links.append(full_link)
            visited_links.add(full_link)
#full_links

In [52]:
marcas = ['Aprilia', 'Bmw', 'Ducati', 'Honda', 'Yamaha', 'Kawasaki', 'Suzuki', 'KTM']

diccionario_rangos = {'Aprilia': 1, 'Bmw': 5, 'Ducati': 11, 'Honda': 20, 'Yamaha': 67, 'Kawasaki': 25, 'Suzuki': 56, 'KTM': 28}

logo_links = []

for marca in marcas:
    rango = diccionario_rangos[marca]
    xpath = f'//*[@id="content"]/div[4]/div[1]/div[{rango}]/a/img'
    logo = driver.find_element(By.XPATH, xpath)
    logo_links.append(logo.get_attribute("src"))

df = pd.DataFrame({'logo_links': logo_links, 'marcas': marcas})
df

,logo_links,marcas
0,https://motos-b60.kxcdn.com/sites/default/files/aprilia.png,Aprilia
1,https://motos-b60.kxcdn.com/sites/default/files/logo_bmw.png,Bmw
2,https://motos-b60.kxcdn.com/sites/default/files/ducati.png,Ducati
3,https://motos-b60.kxcdn.com/sites/default/files/honda.png,Honda
4,https://motos-b60.kxcdn.com/sites/default/files/logo-yamaha.jpg,Yamaha
5,https://motos-b60.kxcdn.com/sites/default/files/kawasaki-logo.jpg,Kawasaki
6,https://motos-b60.kxcdn.com/sites/default/files/suzuki.jpg,Suzuki
7,https://motos-b60.kxcdn.com/sites/default/files/ktm.png,KTM


In [53]:
df_logos = df

In [54]:
df_logos.to_csv('df_logos.csv')